# Custom Shuffle Function

Python uses seeds from computer clock, let's try to do this ourselves.

In [1]:
import numpy as np
import time

To understand how "time" works: https://www.geeksforgeeks.org/python-time-time-method/

In basic terms, your computer has a built in clock that starts from an epoch that you can check from time.gmtime(0). Using time.time() will grab the amount of seconds since the epoch. Since time is constantly moving forward this time.time() will keep changing, so that will help in always have a random number.

In [2]:
print("epoch =", time.gmtime(0))
print("time.time()=", time.time())

epoch = time.struct_time(tm_year=1970, tm_mon=1, tm_mday=1, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=3, tm_yday=1, tm_isdst=0)
time.time()= 1738354467.8584168


Next, actually using time.time() to shuffle an array... 

My idea is that I want to swap two random indices. An issue arises: I need to make sure I am still in range. I can't just use arr[time.time()] as that would be astronomically out range. Trying to divide that by a set number won't always work because you may still not be in range overtime. But, what I can do is use the idea of modulus (%) to make sure I am still in the range of the array. This will basically only consider your remainders as you divide time.time() by *something*. If I can keep these remainders in the range of the original array, we are golden. 

I knew that I was going to use a for loop, and with that I realized I can have "i" that starts at 0 and goes through to the length of my array. I don't want to divide by 0 at first, so I will instead divide time.time() by i+1. For simplicity of understanding the modulus better, I made time.time() into an integer, which will just round up to the nearest integer.
https://www.geeksforgeeks.org/what-is-a-modulo-operator-in-python/


For example:
- say time.time() = 1738350574.6431892 (I got this when checking it randomly once)
- Then we get int(time.time()) = 1738350575 as this rounds up
- We will get then for i = 0: 1738350575%1. This will always = 0
- Then i increases by 1 and you get i=1: 1738350575%2 = 1 (think of it like even, odd, where even = 0, odd = 1)
- One more example for i = 2, 1738350575%3 = 2. (think in terms of remainder+0, 1/3, 2/3 when you divide by 3)

This pattern continues, you will always stay in range since your modulus will never be more than i (modulus i+1 goes from 0 to i)

My very last issue is how I will actually swap the elements. I chose to do a simple two element swap, but I think this could really be anything you want as long as its in range. Doing this means sometimes they won't swap because you could get arr[x] = arr[x] at some points, but with the randomness of time, this won't occur for every x. Notice that sometimes maybe you swapped arr[i] with arr[j], but then j changes, so you may just swap right back. This is okay! It is still going to be random almost every time because again, we are using the randomness of time. With greater sizes of arrays gives greater randomness too:)

In [36]:
#function for shuffling 1D arrays that will use this time.time() function
def shuffle_1D(arr):
    for i in range(0, len(arr)):
        j = int(time.time()) % (i + 1)
        arr[i], arr[j] = arr[j], arr[i]
    return arr

In [37]:
#1D array check:
arr_1D = np.arange(1,10,1)
print("pre shuffled 1D array =", arr_1D)
arr_1D_shuffle = shuffle(arr_1D)
print("shuffled 1D array =", arr_1D_shuffle)

pre shuffled 1D array = [1 2 3 4 5 6 7 8 9]
shuffled 1D array = [3 7 1 9 2 4 5 6 8]


I realized this only worked for 1D arrays, I would need to change a few things if I want it to work with multidimensional arrays, since len(arr) will actually give me the amount of rows instead of what is actually inside of those rows. The first thing I did then was differentiate whether the array thats being passed into my function is 1D or not, I can do this with an if/else statement using len(arr.shape). This will output 1 if its 1D, and anything greater than 1 for multidimensional arrays. So there is my condition! 

Now I needed to loop over the multidimensional array rows. I decided to do a nested for-loop, the first one will tell me which row I am in. I will start at the first row (row 0) and then do the shuffle to that row, then go to the next row, and so on, until there are no more rows. So, I need to be in range of the amount of rows, which I can now get with len(arr) as I ruled out the 1D case. The nested for loop will be almost identical to the 1D array case, except we just have to specify that we are on the specific row to get that index. I used 'a' as my first for-loop variable, so in this case, it would be arr[a][i]. For example, for a = 0, this notes that we are at the first row (arr[0]), and to get the ith value of that row [i], with i iterating over the length of the row (len(arr[a])).

One issue arose... all the shuffles were nearly identical if I didn't already have a random array. This is because time.time() would be acting like a seed at that instant. So, I just needed to make sure that there was a way of changing this for every row just in case. I opted to change time.time() by multiplying it by itself, so we get something crazy random again every time that won't affect the modulus

In [38]:
#this can take in any shape array now
def shuffle(arr):
    if len(arr.shape) > 1:
        for a in range(len(arr)):
            new_time = time.time()**2
            for i in range(0, len(arr[a])):
                j = int(new_time) % (i + 1)
                arr[a][i], arr[a][j] = arr[a][j], arr[a][i]
    else:
        for i in range(0, len(arr)):
            j = int(time.time()) % (i + 1)
            arr[i], arr[j] = arr[j], arr[i]
    return arr

In [40]:
#feel free to change these test arrays to anything to see if it works to shuffle!

#1D array check:
arr_1D = np.arange(1,10,1)
print("pre shuffled 1D array =", arr_1D)
arr_1D_shuffle = shuffle(arr_1D)
print("shuffled 1D array =", arr_1D_shuffle)

#2D array check:
test_arr = np.arange(1,10,1)
arr1 = np.arange(1,10,1)
arr2 = np.arange(1,10,1)
arr_2D = np.array([arr1, arr2])
print("pre shuffled 2D array =", arr_2D)
arr_2D_shuffle = shuffle(arr_2D)
print("shuffled 2D array =", arr_2D_shuffle)

#shuffling from the random array creator
arr_random = np.random.rand(4, 7)
print("pre shuffled random array =", arr_random)
arr_random_shuffle = shuffle(arr_random)
print("shuffled random array =", arr_random_shuffle)

pre shuffled 1D array = [1 2 3 4 5 6 7 8 9]
shuffled 1D array = [7 1 6 5 3 9 8 2 4]
pre shuffled 2D array = [[1 2 3 4 5 6 7 8 9]
 [1 2 3 4 5 6 7 8 9]]
shuffled 2D array = [[9 1 2 7 3 4 5 6 8]
 [9 5 2 7 1 4 3 6 8]]


TypeError: only length-1 arrays can be converted to Python scalars